In [15]:
import obd

connection = obd.OBD("/dev/pts/2") # auto-connects to USB or RF port

cmd = obd.commands.SPEED # select an OBD command (sensor)

response = connection.query(cmd) # send the command, and parse the response

print(response.value) # returns unit-bearing values thanks to Pint
print(response.value.to("mph")) # user-friendly unit conversions

29.0 kph
18.01976457488269 mph


In [17]:
# To see the definition of the speed PID (as defined by OBD2)
# see https://en.wikipedia.org/wiki/OBD-II_PIDs

c = obd.OBDCommand("mySPEED",            # name
               "Vehicle Speed",          # description
               b"010D",                  # OBD PID 01 (group) 0D (sensor)
               3,                        # number of return bytes to expect
               obd.decoders.uas(0x09),   # decoding function
               obd.protocols.ECU.ENGINE, # (optional) ECU filter
               True)                     # (optional) allow a "01" to be added for speed

# The author provides many built-in commands. To see how these commands are defined
# see https://github.com/brendan-w/python-OBD/blob/master/obd/commands.py

response = connection.query(cmd) # send the command, and parse the response

print(response.value) # returns unit-bearing values thanks to Pint
print(response.value.to("mph")) # user-friendly unit conversions

41.0 kph
25.476218881730695 mph


In [ ]:
def bytes_to_int(bs):
    """ converts a big-endian byte array into a single integer """
    v = 0
    p = 0
    for b in reversed(bs):
        v += b * (2 ** p)
        p += 8
    return v

def speed_decoder(messages):
    _bytes = messages[0].data[2:]
    value = bytes_to_int(_bytes)
    return Unit.Quantity(value, "kph")